In [1]:
import sys
import os
sys.path.append(os.path.abspath("../src"))

from data_manipulation_func import load_epex_data, clean_data
print('Successful import')
df = load_epex_data('../data/spot_prices_2025_raw.csv')
df = clean_data(df)
df.head()

Successful import


,MTU (CET/CEST),month,day,hour,Day-ahead Price (EUR/MWh)
0,2025-01-01 00:00:00,1,1,0,118.46
1,2025-01-01 01:00:00,1,1,1,129.07
2,2025-01-01 02:00:00,1,1,2,121.10
3,2025-01-01 03:00:00,1,1,3,94.28
4,2025-01-01 04:00:00,1,1,4,63.69
